<a href="https://colab.research.google.com/github/xiaoyanhouston/job_recommendation/blob/main/job_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
MOUNTPOINT = '/content/drive'
drive.mount(MOUNTPOINT)

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install python-dotenv langchain openai pypdf pymupdf  tiktoken chromadb PyPDF2 langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.9/309.9 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
file_path = "/content/drive/MyDrive/My_project/Astoria_Pro"

In [ ]:

from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA,LLMChain,SimpleSequentialChain,SequentialChain
import textwrap
import os
# from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
# from openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader
from IPython.display import display, HTML




In [ ]:
# API Keys for OPENAI
load_dotenv('/content/drive/MyDrive/My_project/my_key/.env')
LLM_KEY=os.getenv("OPENAI_API_KEY")


test turbo 3.5 and results are not reasonable


In [ ]:
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo',
    api_key=LLM_KEY
)

test match case

In [ ]:
reloader = PyPDFLoader(f"{file_path}/resume/Resume_Stella_HR_DS.pdf")
retext = reloader.load_and_split()
jbloader = PyPDFLoader(f"{file_path}/jobs/006JD.pdf")
jobtext = jbloader.load_and_split()

In [ ]:
resume_template="""Write a comprehensive summary of the resume in:
{resume}, be sure include years of experience, education, skills and summarize within 200 words"""
resume_prompt_template=PromptTemplate(template=resume_template,
                                     input_variables=["resume"] )

resume_chain = LLMChain(llm=llm, prompt=resume_prompt_template,output_key="res_sum")
job_template="""Write a summary of the job {job} only if it matches the resume {res_sum},
if matches, be sure to include job title, company name, Location, salarly,technical skills,
 minimum requirements and summarize the job part relevant to the resume,

Title:
Company Name:
Location:
Salary:
Technical skills:
Requirement:
Job Summary:

if the job doesn't match the resume, skip all the details mentioned above,
just say the job does not match your experience , do not come up answer.

Be sure format result itermized as HTML so it could be displayed on website.
"""
job_prompt_template=PromptTemplate(template=job_template,
                                     input_variables=["job","res_sum"] )

job_chain = LLMChain(llm=llm, prompt=job_prompt_template)

ss_chain= SequentialChain(chains=[resume_chain, job_chain],input_variables=["resume","job"])

In [ ]:
display(HTML(ss_chain.run(resume=retext,job=jobtext)))

test non match case

In [ ]:
reloader = PyPDFLoader(f"{file_path}/resume/Resume_Stella_HR_DS.pdf")
retext = reloader.load_and_split()
jbloader = PyPDFLoader(f"{file_path}/jobs/001JD.pdf")
jobtext = jbloader.load_and_split()

In [ ]:
resume_template="""Write a comprehensive summary of the resume in:
{resume}, be sure include years of experience, education, skills and summarize within 200 words"""
resume_prompt_template=PromptTemplate(template=resume_template,
                                     input_variables=["resume"] )

resume_chain = LLMChain(llm=llm, prompt=resume_prompt_template,output_key="res_sum")
job_template="""Write a summary of the job {job} only if it matches the resume {res_sum},
if matches, be sure to include job title, company name, Location, salarly,technical skills,
 minimum requirements and summarize the job part relevant to the resume,

Title:
Company Name:
Location:
Salary:
Technical skills:
Requirement:
Job Summary:

if the job doesn't match the resume, skip all the details mentioned above,
just say the job does not match your experience , do not come up answer.

Be sure format result itermized as HTML so it could be displayed on website.
"""
job_prompt_template=PromptTemplate(template=job_template,
                                     input_variables=["job","res_sum"] )

job_chain = LLMChain(llm=llm, prompt=job_prompt_template)

ss_chain= SequentialChain(chains=[resume_chain, job_chain],input_variables=["resume","job"])

In [ ]:
display(HTML(ss_chain.run(resume=retext,job=jobtext)))

Test turbo 4.0

In [ ]:
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4-turbo',
    api_key=LLM_KEY
)

Test match case

In [ ]:

reloader = PyPDFLoader(f"{file_path}/resume/Resume_Stella_HR_DS.pdf")
retext = reloader.load_and_split()
jbloader = PyPDFLoader(f"{file_path}/jobs/006JD.pdf")
jobtext = jbloader.load_and_split()

In [ ]:
resume_template="""Write a comprehensive summary of the resume in:
{resume}, be sure include years of experience, education, skills and summarize within 200 words"""
resume_prompt_template=PromptTemplate(template=resume_template,
                                     input_variables=["resume"] )

resume_chain = LLMChain(llm=llm, prompt=resume_prompt_template,output_key="res_sum")
job_template="""Write a summary of the job {job} only if it matches the resume {res_sum},
if matches, be sure to include job title, company name, Location, salarly,technical skills,
 minimum requirements and summarize the job part relevant to the resume,

Title:
Company Name:
Location:
Salary:
Technical skills:
Requirement:
Job Summary:

if the job doesn't match the resume, skip all the details mentioned above,
just say the job does not match your experience , do not come up answer.

Be sure format result itermized as HTML so it could be displayed on website.
"""
job_prompt_template=PromptTemplate(template=job_template,
                                     input_variables=["job","res_sum"] )

job_chain = LLMChain(llm=llm, prompt=job_prompt_template)

ss_chain= SequentialChain(chains=[resume_chain, job_chain],input_variables=["resume","job"])

In [ ]:
display(HTML(ss_chain.run(resume=retext,job=jobtext)))

Test non match case


In [ ]:

reloader = PyPDFLoader(f"{file_path}/resume/Resume_Stella_HR_DS.pdf")
retext = reloader.load_and_split()
jbloader = PyPDFLoader(f"{file_path}/jobs/001JD.pdf")
jobtext = jbloader.load_and_split()

In [ ]:
resume_template="""Write a comprehensive summary of the resume in:
{resume}, be sure include years of experience, education, skills and summarize within 200 words"""
resume_prompt_template=PromptTemplate(template=resume_template,
                                     input_variables=["resume"] )

resume_chain = LLMChain(llm=llm, prompt=resume_prompt_template,output_key="res_sum")
job_template="""Write a summary of the job {job} only if it matches the resume {res_sum},
if matches, be sure to include job title, company name, Location, salarly,technical skills,
 minimum requirements and summarize the job part relevant to the resume,

Title:
Company Name:
Location:
Salary:
Technical skills:
Requirement:
Job Summary:

if the job doesn't match the resume, skip all the details mentioned above,
just say the job does not match your experience , do not come up answer.

Be sure format result itermized as HTML so it could be displayed on website.
"""
job_prompt_template=PromptTemplate(template=job_template,
                                     input_variables=["job","res_sum"] )

job_chain = LLMChain(llm=llm, prompt=job_prompt_template)

ss_chain= SequentialChain(chains=[resume_chain, job_chain],input_variables=["resume","job"])

In [ ]:
display(HTML(ss_chain.run(resume=retext,job=jobtext)))

define a function to do the summarization for all jobs

In [ ]:
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4-turbo',
    api_key=LLM_KEY
)

In [ ]:
def job_summary(llm,resume_path,job_path):
  reloader = PyPDFLoader(resume_path)
  retext = reloader.load_and_split()
  jbloader = PyPDFLoader(job_path)
  jobtext = jbloader.load_and_split()
  resume_template="""Write a comprehensive summary of the resume in:
  {resume}, be sure include years of experience, education, skills and summarize within 200 words"""
  resume_prompt_template=PromptTemplate(template=resume_template,
                                      input_variables=["resume"] )

  resume_chain = LLMChain(llm=llm, prompt=resume_prompt_template,output_key="res_sum")
  job_template="""Write a summary of the job {job} only if it matches the resume {res_sum},
  if matches, be sure to include job title, company name, Location, salarly,technical skills,
  minimum requirements and summarize the job part relevant to the resume,

  Title:
  Company Name:
  Location:
  Salary:
  Technical skills:
  Requirement:
  Job Summary:

  if the job doesn't match the resume, skip all the details mentioned above,
  just say the job does not match your experience , do not come up answer.

  Be sure format result itermized as HTML so it could be displayed on website.
  """
  job_prompt_template=PromptTemplate(template=job_template,
                                      input_variables=["job","res_sum"] )

  job_chain = LLMChain(llm=llm, prompt=job_prompt_template)

  ss_chain= SequentialChain(chains=[resume_chain, job_chain],input_variables=["resume","job"])
  display(HTML(ss_chain.run(resume=retext,job=jobtext)))

In [ ]:
# Define the folder path
folder_path = f"{file_path}/jobs"

# Get a list of all files and directories in the specified folder
files_and_dirs = os.listdir(folder_path)

# Filter out only the files from the list
files = [os.path.join(folder_path, f) for f in files_and_dirs]


In [ ]:
res_path=f"{file_path}/resume/Resume_Stella_HR_DS.pdf"

for i in files:
  print(i)
  job_summary(llm=llm, resume_path=res_path ,job_path=i)
  print("----------------------------------------------")

/content/drive/MyDrive/My_project/Astoria_Pro/jobs/001JD.pdf


----------------------------------------------
/content/drive/MyDrive/My_project/Astoria_Pro/jobs/002JD.pdf


----------------------------------------------
/content/drive/MyDrive/My_project/Astoria_Pro/jobs/003JD.pdf


----------------------------------------------
/content/drive/MyDrive/My_project/Astoria_Pro/jobs/004JD.pdf


----------------------------------------------
/content/drive/MyDrive/My_project/Astoria_Pro/jobs/005JD.pdf


----------------------------------------------
/content/drive/MyDrive/My_project/Astoria_Pro/jobs/007JD.pdf


----------------------------------------------
/content/drive/MyDrive/My_project/Astoria_Pro/jobs/006JD.pdf


----------------------------------------------
/content/drive/MyDrive/My_project/Astoria_Pro/jobs/008JD.pdf


----------------------------------------------


In [ ]:
files

['/content/drive/MyDrive/My_project/Astoria_Pro/jobs/001JD.pdf',
 '/content/drive/MyDrive/My_project/Astoria_Pro/jobs/002JD.pdf',
 '/content/drive/MyDrive/My_project/Astoria_Pro/jobs/003JD.pdf',
 '/content/drive/MyDrive/My_project/Astoria_Pro/jobs/004JD.pdf',
 '/content/drive/MyDrive/My_project/Astoria_Pro/jobs/005JD.pdf',
 '/content/drive/MyDrive/My_project/Astoria_Pro/jobs/007JD.pdf',
 '/content/drive/MyDrive/My_project/Astoria_Pro/jobs/006JD.pdf',
 '/content/drive/MyDrive/My_project/Astoria_Pro/jobs/008JD.pdf']

Assess a sepecific job for the talent

In [ ]:
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4-turbo',
    api_key=LLM_KEY
)

In [ ]:
reloader = PyPDFLoader(f"{file_path}/resume/Resume_Stella_HR_DS.pdf")
retext = reloader.load_and_split()
jbloader = PyPDFLoader(f"{file_path}/jobs/005JD.pdf")
jobtext = jbloader.load_and_split()

In [ ]:
resume_template="""Write a comprehensive summary of the resume in:
{resume}, be sure include years of experience, education, skills and summarize within 200 words"""
resume_prompt_template=PromptTemplate(template=resume_template,
                                     input_variables=["resume"] )

resume_chain = LLMChain(llm=llm, prompt=resume_prompt_template,output_key="res_sum")
job_template="""you are a career expert need to assess the job {job} for the candidate
with the resume {res_sum} by the following 2 steps:
1. summarize the part in the job description relevant to the resume
2. provide assessment summary to the candidate where he/she need to improve
show the results as the following 2 items,
Relevant summary:
Need to improve:

Be sure format result itermized as HTML so it could be displayed on website.
"""
job_prompt_template=PromptTemplate(template=job_template,
                                     input_variables=["job","res_sum"] )

job_chain = LLMChain(llm=llm, prompt=job_prompt_template)

ss_chain= SequentialChain(chains=[resume_chain, job_chain],input_variables=["resume","job"])

In [ ]:
display(HTML(ss_chain.run(resume=retext,job=jobtext)))

Write a cover letter

In [ ]:
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4-turbo',
    api_key=LLM_KEY
)

In [ ]:
reloader = PyPDFLoader(f"{file_path}/resume/Resume_Stella_HR_DS.pdf")
retext = reloader.load_and_split()
jbloader = PyPDFLoader(f"{file_path}/jobs/005JD.pdf")
jobtext = jbloader.load_and_split()

In [ ]:
from datetime import date
today= date.today()

In [ ]:
resume_template="""Write a comprehensive summary of the resume in:
{resume}, be sure include years of experience, education, skills and summarize within 400 words"""
resume_prompt_template=PromptTemplate(template=resume_template,
                                     input_variables=["resume"] )

resume_chain = LLMChain(llm=llm, prompt=resume_prompt_template,output_key="res_sum")
job_template="""you are a career expert need to write a cover letter for the candidate
with the resume {res_sum} and the candidate is applying the job {job}.
make sure to highlight the relevant experience to the job. Use 'Cover Letter' as title in the middle,
use {today} as date to the left and  start the letter body with Dear Hiring Manager. Order them as :
Title
Date
Letter Body
.

Be sure format result itermized as HTML so it could be displayed on website.
"""
job_prompt_template=PromptTemplate(template=job_template,
                                     input_variables=["job","res_sum","today"] )

job_chain = LLMChain(llm=llm, prompt=job_prompt_template)

ss_chain= SequentialChain(chains=[resume_chain, job_chain],input_variables=["resume","job","today"])

In [ ]:
display(HTML(ss_chain.run(resume=retext,job=jobtext,today=today)))

Test RAG


In [ ]:
# Reading all the pages of the PDF file using PyPDFLoader
text=""
#Docx2txtLoader loads the Document
loader=DirectoryLoader(f"{file_path}/jobs/",
                       glob = '*.pdf',
                       loader_cls=PyPDFLoader)
#Load Documents and split into chunks
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=3000, chunk_overlap = 100)
text = text_splitter.split_documents(docs)


In [ ]:
# Embed and store the job description files
#  store the vectors into vectore store
vec_dir = f"{file_path}/vec"

embeddings = OpenAIEmbeddings(openai_api_key=LLM_KEY)

# vecdb = Chroma.from_documents(documents=text,
#                               embedding=embeddings,
#                               persist_directory= vec_dir)
# # store vec to disk
# vecdb.persist()
# vecdb = None

# read vectors from local now.
vecdb = Chroma(persist_directory=vec_dir,
                  embedding_function=embeddings)

retriever = vecdb.as_retriever(search_kwargs={"k": 3})

In [ ]:
import PyPDF2
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)
        for page_number in range(num_pages):
            page = pdf_reader.pages[page_number]
            text += page.extract_text()
    return text


In [ ]:
resume = extract_text_from_pdf(f"{file_path}/resume/Resume_Stella_HR_DS.pdf")
query =  """you are a career expert, your job is to rank the matched jobs from context delimited by <ctx></ctx> for the resume below the context:
<ctx>
{context},
</ctx>
\n
""" + resume + """
\n
short list jobs which is good fit based on skills,education and work experience mwntioned in it?
also provide the job company name,title,salarly,job summary,put them as python list, If there is no match, just say no match.
{question}, be sure to put the results as HTML format so it could be used on website.
Answer: let's think step by step"""
prompt= PromptTemplate(
    input_variables=['context','question'],
    template= query

)


In [ ]:
llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-4-turbo',
    api_key=LLM_KEY
)

qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type="stuff",
                                       retriever=retriever,
                                       chain_type_kwargs={'prompt': prompt}
                                      #  ,
                                      #  return_source_documents=True
                                       )

def print_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [ ]:
display(HTML(qa_chain.run("give me the top 3 jobs")))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
